## Importing libraries

In [1]:
import numpy as np # for linear algebra , math equations
import math # for math
import pandas as pd # for dataframes
from sklearn.feature_extraction.text import TfidfVectorizer # to vectorize the tf_idf values
import os # read files from folder
import natsort as nt #to sort files in natural number order
from nltk.tokenize import word_tokenize # to make a list of strings
from nltk.corpus import stopwords # to remove unwanted words (very common)
import warnings
warnings.filterwarnings("ignore")

# PART 1

### a) Read 10 files

In [2]:
path = 'C:/Users/Eyad/Desktop/AI course,Data Science/projects/IR CS/documents/'
os.chdir(path)

In [3]:
files = nt.natsorted(os.listdir())
files

['1.txt',
 '2.txt',
 '3.txt',
 '4.txt',
 '5.txt',
 '6.txt',
 '7.txt',
 '8.txt',
 '9.txt',
 '10.txt']

In [4]:
dicts = {}
keys = range(1,len(files)+1)
values = []
for file in files:
    with open(file , 'r', encoding='utf-8') as f:
        values.append(f.read())
for i in keys:
        dicts[i] = values[i-1]
print(dicts)

{1: 'antony brutus caeser cleopatra mercy worser', 2: 'antony brutus caeser calpurnia ', 3: 'mercy worser', 4: 'brutus caeser mercy worser', 5: 'caeser mercy worser', 6: 'antony caeser mercy ', 7: 'angels fools fear in rush to tread where', 8: 'angels fools fear in rush to tread where', 9: 'angels fools in rush to tread where', 10: 'fear in rush to tread where fools'}


In [5]:
def append_new_doc(doc,filenumber): # to be more dynamic
    with open(doc , 'r', encoding='utf-8') as f:
        dicts[filenumber] = f.read()

### b ) Tokenization

In [6]:
def tokenize_query(query): 
    query = word_tokenize(query)
    return query

In [7]:
for i in dicts:
    dicts[i] = tokenize_query(dicts[i])
    print(dicts[i])

['antony', 'brutus', 'caeser', 'cleopatra', 'mercy', 'worser']
['antony', 'brutus', 'caeser', 'calpurnia']
['mercy', 'worser']
['brutus', 'caeser', 'mercy', 'worser']
['caeser', 'mercy', 'worser']
['antony', 'caeser', 'mercy']
['angels', 'fools', 'fear', 'in', 'rush', 'to', 'tread', 'where']
['angels', 'fools', 'fear', 'in', 'rush', 'to', 'tread', 'where']
['angels', 'fools', 'in', 'rush', 'to', 'tread', 'where']
['fear', 'in', 'rush', 'to', 'tread', 'where', 'fools']


In [8]:
#Test
q = 'Hello im eyad'
q = tokenize_query(q)
q

['Hello', 'im', 'eyad']

## c) Apply stop words (except: in , to,where)

In [9]:
stopwords_edited = stopwords.words('english')
stopwords_edited.remove('in')
stopwords_edited.remove('to')
stopwords_edited.remove('where')

In [10]:
def remove_stop_words(query):
    query = [word for word in query if not word in stopwords_edited]
    return query

In [11]:
for i in dicts:
    dicts[i] = remove_stop_words(dicts[i])
    print(dicts[i])

['antony', 'brutus', 'caeser', 'cleopatra', 'mercy', 'worser']
['antony', 'brutus', 'caeser', 'calpurnia']
['mercy', 'worser']
['brutus', 'caeser', 'mercy', 'worser']
['caeser', 'mercy', 'worser']
['antony', 'caeser', 'mercy']
['angels', 'fools', 'fear', 'in', 'rush', 'to', 'tread', 'where']
['angels', 'fools', 'fear', 'in', 'rush', 'to', 'tread', 'where']
['angels', 'fools', 'in', 'rush', 'to', 'tread', 'where']
['fear', 'in', 'rush', 'to', 'tread', 'where', 'fools']


In [12]:
all_words = []
for doc in dicts:
    for term in dicts[doc]:
        all_words.append(term)
all_words = sorted(all_words)

In [13]:
#test
q = 'Where are you at the to in from from in egypt'
q = tokenize_query(q)
q = remove_stop_words(q)
q

['Where', 'to', 'in', 'in', 'egypt']

# PART 2

### a) Building positional index

In [14]:
doc_no = 1
pos_idx = {}
def positional_indexing(doc):
    global doc_no
    global pos_idx
     #looping in each word in document
    if term in pos_idx:  #check if term is in dictionary already
        pos_idx[term][0] += 1    # if term is in dictionary the frequency of it will increase by 1
        if doc_no in pos_idx[term][1]: # if it was already in the same document
            pos_idx[term][1][doc_no].append(pos)   # it will append the other position index
        else:
            pos_idx[term][1][doc_no] = [pos] # if not it will create the first position it appeared in the document
    else:

        pos_idx[term]=[]   # if term is not in the document

        pos_idx[term].append(1) # it will append number one as it appeared for first time

        pos_idx[term].append({})  # it will append a dictionary next to the list to put term positions

        pos_idx[term][1][doc_no] = [pos] #it will put the position as a value in the key of document number

     # increase document number to change keys to be next in the next loop

In [15]:
doc_no = 1
pos_idx = {}
for doc in dicts: 
    for pos,term in enumerate(dicts[doc]):
        positional_indexing(doc)
    doc_no +=1
print(pos_idx)

{'antony': [3, {1: [0], 2: [0], 6: [0]}], 'brutus': [3, {1: [1], 2: [1], 4: [0]}], 'caeser': [5, {1: [2], 2: [2], 4: [1], 5: [0], 6: [1]}], 'cleopatra': [1, {1: [3]}], 'mercy': [5, {1: [4], 3: [0], 4: [2], 5: [1], 6: [2]}], 'worser': [4, {1: [5], 3: [1], 4: [3], 5: [2]}], 'calpurnia': [1, {2: [3]}], 'angels': [3, {7: [0], 8: [0], 9: [0]}], 'fools': [4, {7: [1], 8: [1], 9: [1], 10: [6]}], 'fear': [3, {7: [2], 8: [2], 10: [0]}], 'in': [4, {7: [3], 8: [3], 9: [2], 10: [1]}], 'rush': [4, {7: [4], 8: [4], 9: [3], 10: [2]}], 'to': [4, {7: [5], 8: [5], 9: [4], 10: [3]}], 'tread': [4, {7: [6], 8: [6], 9: [5], 10: [4]}], 'where': [4, {7: [7], 8: [7], 9: [6], 10: [5]}]}


### b) Allow users to write queries

In [16]:
def return_matched_docs_ix(q):
    pos_idx_list = [[] for i in range(len(pos_idx))]
    for w in q: # looping every word in query
        
        try:
            
            for k in pos_idx[w][1].keys(): # looping key in keys in each term

                if pos_idx_list[k-1] != []:

                    if pos_idx_list[k-1][-1] == pos_idx[w][1][k][0]-1:
                        pos_idx_list[k-1].append(pos_idx[w][1][k][0])

                else:
                        pos_idx_list[k-1].append(pos_idx[w][1][k][0])

            
            for ix , lists in enumerate(pos_idx_list):
                if len(q) == len(lists):
                    print("Matched in doc number:" , ix+1)
        except KeyError:
            print("No matched document -> invalid input")

In [17]:
# test
q = "fools fear in"
q = tokenize_query(q)
q = remove_stop_words(q)
print(q)
return_matched_docs_ix(q)

['fools', 'fear', 'in']
Matched in doc number: 7
Matched in doc number: 8


# Part 3

### a) Term frequency

In [18]:
def get_term_freq(doc): # TF = total time word appeared in document
    words_found = dict.fromkeys(all_words , 0) #dictionary with all words frequencies are = 0
    for word in doc: # looping in each word in the document
        words_found[word] += 1 # if we found the word , we increase the value (0) to be (1)
    return words_found   # returning the new dictionary after getting all frequency

In [19]:
tf = pd.DataFrame()
for i in range(1 , len(dicts)+1):
    tf[i] =  pd.DataFrame(
                            get_term_freq(dicts[i]).values() ,
                            index = get_term_freq(dicts[i]).keys()
                         )
    
tf.columns = [f"DOC_{i}" for i in range(1,11)]
tf.style.background_gradient(cmap = "Blues")


,DOC_1,DOC_2,DOC_3,DOC_4,DOC_5,DOC_6,DOC_7,DOC_8,DOC_9,DOC_10
angels,0,0,0,0,0,0,1,1,1,0
antony,1,1,0,0,0,1,0,0,0,0
brutus,1,1,0,1,0,0,0,0,0,0
caeser,1,1,0,1,1,1,0,0,0,0
calpurnia,0,1,0,0,0,0,0,0,0,0
cleopatra,1,0,0,0,0,0,0,0,0,0
fear,0,0,0,0,0,0,1,1,0,1
fools,0,0,0,0,0,0,1,1,1,1
in,0,0,0,0,0,0,1,1,1,1
mercy,1,0,1,1,1,1,0,0,0,0


##### Weighted term frequency

In [20]:
def get_weighted_term_freq(x): # log(tf)+1    ## to decay the difference gap between documents 
    try:
        return math.log(x)+1
    except ValueError:
        return 0

In [21]:
wtf = tf.copy()

In [22]:
for i in range(1,len(dicts)+1):
    wtf[f"DOC_{i}"] = wtf[f"DOC_{i}"].apply(get_weighted_term_freq) #apply wtf to each cell in tf matrix
wtf.astype(int).style.background_gradient(cmap = "Blues")

,DOC_1,DOC_2,DOC_3,DOC_4,DOC_5,DOC_6,DOC_7,DOC_8,DOC_9,DOC_10
angels,0,0,0,0,0,0,1,1,1,0
antony,1,1,0,0,0,1,0,0,0,0
brutus,1,1,0,1,0,0,0,0,0,0
caeser,1,1,0,1,1,1,0,0,0,0
calpurnia,0,1,0,0,0,0,0,0,0,0
cleopatra,1,0,0,0,0,0,0,0,0,0
fear,0,0,0,0,0,0,1,1,0,1
fools,0,0,0,0,0,0,1,1,1,1
in,0,0,0,0,0,0,1,1,1,1
mercy,1,0,1,1,1,1,0,0,0,0


### b) Inverse document frequency

In [23]:
idf_df = pd.DataFrame(index = get_term_freq(dicts[1]).keys() , columns=["df" , "idf"])

In [24]:
for i in idf_df.index: ## inverse document frequency -> used to decay the times documents are important to another document
    idf_df['df'][i] = pos_idx[i][0] #frequency of each term in all documents
    idf_df['idf'][i] = np.log10( len(dicts.keys()) / float(pos_idx[i][0]) ) # inverse document formula for each word

In [25]:
# type casting just to apply the colors
idf_df["df"] = idf_df["df"].astype(int)
idf_df["idf"] = idf_df["idf"].astype(float)

In [26]:
idf_df.style.background_gradient(cmap = "Blues" , axis= 0)

,df,idf
angels,3,0.522879
antony,3,0.522879
brutus,3,0.522879
caeser,5,0.301030
calpurnia,1,1.000000
cleopatra,1,1.000000
fear,3,0.522879
fools,4,0.397940
in,4,0.397940
mercy,5,0.301030


### c) Term frequency - Inverse document frequency

In [27]:
tf_idf = tf.multiply(idf_df["idf"] , axis = 0) # axis = 0 to multiply rows not columns

In [28]:
tf_idf.style.background_gradient(cmap = "Blues" , axis= 0)

,DOC_1,DOC_2,DOC_3,DOC_4,DOC_5,DOC_6,DOC_7,DOC_8,DOC_9,DOC_10
angels,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.522879,0.522879,0.522879,0.000000
antony,0.522879,0.522879,0.000000,0.000000,0.000000,0.522879,0.000000,0.000000,0.000000,0.000000
brutus,0.522879,0.522879,0.000000,0.522879,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
caeser,0.301030,0.301030,0.000000,0.301030,0.301030,0.301030,0.000000,0.000000,0.000000,0.000000
calpurnia,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
cleopatra,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
fear,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.522879,0.522879,0.000000,0.522879
fools,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.397940,0.397940,0.397940,0.397940
in,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.397940,0.397940,0.397940,0.397940
mercy,0.301030,0.000000,0.301030,0.301030,0.301030,0.301030,0.000000,0.000000,0.000000,0.000000


### Document length & Normalized tf_idf

##### Document length

In [29]:
doc_length = pd.DataFrame()

In [30]:
def get_doc_length(col):
    return np.sqrt(tf_idf[col].apply(lambda x: x**2).sum())

In [31]:
for col in tf_idf.columns:
    doc_length.loc[ 0 , col + '_length'] = get_doc_length(col)

In [32]:
doc_length = doc_length.T

In [33]:
doc_length.columns = ['']

In [34]:
doc_length.style.background_gradient(cmap = 'Blues')

,
DOC_1_length,1.373462
DOC_2_length,1.279618
DOC_3_length,0.498974
DOC_4_length,0.782941
DOC_5_length,0.582747
DOC_6_length,0.674270
DOC_7_length,1.223496
DOC_8_length,1.223496
DOC_9_length,1.106137
DOC_10_length,1.106137


##### Normalized term freq inverse doc freq

In [35]:
normalized_tfidf = pd.DataFrame()

In [36]:
def get_normalized_tf_idf(col, x):
    try:
        return x / doc_length.loc[col + '_length'].values[0] # x -> cell
    except ZeroDivisionError:
        return 0

In [37]:
for col in tf_idf.columns:
    normalized_tfidf[col] = tf_idf[col].apply(lambda x: get_normalized_tf_idf(col , x))

In [38]:
normalized_tfidf.style.background_gradient(cmap = 'Blues' , axis= 0)

,DOC_1,DOC_2,DOC_3,DOC_4,DOC_5,DOC_6,DOC_7,DOC_8,DOC_9,DOC_10
angels,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.427365,0.427365,0.472707,0.000000
antony,0.380701,0.408621,0.000000,0.000000,0.000000,0.775474,0.000000,0.000000,0.000000,0.000000
brutus,0.380701,0.408621,0.000000,0.667839,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
caeser,0.219176,0.235250,0.000000,0.384486,0.516570,0.446453,0.000000,0.000000,0.000000,0.000000
calpurnia,0.000000,0.781483,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
cleopatra,0.728087,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
fear,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.427365,0.427365,0.000000,0.472707
fools,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.325248,0.325248,0.359756,0.359756
in,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.325248,0.325248,0.359756,0.359756
mercy,0.219176,0.000000,0.603298,0.384486,0.516570,0.446453,0.000000,0.000000,0.000000,0.000000


### d) Cosine similarity & Ranking documents

In [39]:
docs_cos = list(dicts.values()) 

In [40]:
for i in range(len(docs_cos)):
    docs_cos[i] = ' '.join(docs_cos[i])

In [41]:
vectorizer = TfidfVectorizer()

In [42]:
df = vectorizer.fit_transform(docs_cos).T.toarray() #dataframe for vectorizer

In [43]:
df = pd.DataFrame(df , index= vectorizer.get_feature_names())

In [44]:
df.style.background_gradient(cmap = 'Blues' , axis = 0)

,0,1,2,3,4,5,6,7,8,9
angels,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.385109,0.385109,0.417294,0.000000
antony,0.412627,0.474292,0.000000,0.000000,0.000000,0.662993,0.000000,0.000000,0.000000,0.000000
brutus,0.412627,0.474292,0.000000,0.571154,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
caeser,0.329457,0.378692,0.000000,0.456030,0.555563,0.529358,0.000000,0.000000,0.000000,0.000000
calpurnia,0.000000,0.637721,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
cleopatra,0.554808,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
fear,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.385109,0.385109,0.000000,0.417294
fools,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.342389,0.342389,0.371004,0.371004
in,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.342389,0.342389,0.371004,0.371004
mercy,0.329457,0.000000,0.668165,0.456030,0.555563,0.529358,0.000000,0.000000,0.000000,0.000000


In [45]:
def get_relevant_docs(q, df): #query , dataframe(vectorizer)
    
    print("query is :", q)
    q = tokenize_query(q)
    q = ' '.join(remove_stop_words(q))
    print("query read as :", q)
    print("\n------------------\n"*3)
    
    q = [q]
    q_vec = vectorizer.transform(q).toarray().reshape(df.shape[0])
    sim = {}
    for i in range(10):
        sim[i] = np.dot(df.loc[:, i].values, q_vec) / np.linalg.norm(df.loc[:, i]) * np.linalg.norm(q_vec)
    sim_sorted = sorted(sim.items(), key=lambda x: x[1], reverse=True)
    
    print('\033[1m' + "Most relevant documents:\n")
    returned_docs = []
    print("Cosine similarity: (Query , Document number) ->(Score) : \n")
    for k, v in sim_sorted: #key , value
        if v != 0.0:
            returned_docs.append(k+1)
            print(f"({' '.join(q)} , {k+1}) -> {v}")
            print(f"Document: {' '.join(dicts[k+1])}" , end='\n\n')
    print("Returned documents: " , returned_docs)
    
    return returned_docs

In [46]:
def calc_query_data(q_entered , ret_docs):
    q_entered = [word for word in q_entered.split() if word in list(tf.index)]
    q_entered = ' '.join(q_entered)
    q = pd.DataFrame(index=tf.index)
    q['tf'] = [ 1 if i in q_entered.split() else 0 for i in list(tf.index) ]
    q["wtf"] = [ get_weighted_term_freq(i) for i in q["tf"] ] 
    q["idf"] =  q["wtf"] * idf_df["idf"]
    q['tfidf'] = q['tf'] * q['idf']
    q["norm"] = [(float(q["idf"].iloc[i]) / np.sqrt(sum(q['idf'].values**2))) for i in range(len(q))]


    prod = normalized_tfidf.multiply(q['wtf'] , axis = 0).multiply(q['norm'] , axis = 0)
    prod = prod.loc[q_entered.split() , [f"DOC_{i}" for i in ret_docs]]
    prod = prod.append(prod.sum(numeric_only=True).rename("Sum"))
    for col in prod.columns:
        if 0 in prod[col].values:
            prod.drop( col , axis = 1 ,inplace = True)

    q_len = np.sqrt(sum([x**2 for x in q["idf"].loc[q_entered.split()]]))

    return [  q.loc[q_entered.split() , :] , q_len , prod]

## Search engine

In [47]:
query = input()

fools fear in


In [48]:
get_docs = get_relevant_docs(query, df)

query is : fools fear in
query read as : fools fear in

------------------

------------------

------------------

Most relevant documents:

Cosine similarity: (Query , Document number) ->(Score) : 

(fools fear in , 10) -> 0.6703901383545914
Document: fear in rush to tread where fools

(fools fear in , 7) -> 0.618683750721479
Document: angels fools fear in rush to tread where

(fools fear in , 8) -> 0.618683750721479
Document: angels fools fear in rush to tread where

(fools fear in , 9) -> 0.4106392911359427
Document: angels fools in rush to tread where

Returned documents:  [10, 7, 8, 9]


# ----------------

In [49]:
# query data
calc_query_data(query , get_docs)[0].style.background_gradient(cmap = "Reds" , axis= 0)

,tf,wtf,idf,tfidf,norm
fools,1,1.000000,0.397940,0.397940,0.518023
fear,1,1.000000,0.522879,0.522879,0.680664
in,1,1.000000,0.397940,0.397940,0.518023


In [50]:
## query length
calc_query_data(query , get_docs)[1]

0.7681892236094847

In [51]:
## product
calc_query_data(query , get_docs)[2].style.background_gradient(cmap = "Reds" , axis= 1)

,DOC_10,DOC_7,DOC_8
fools,0.186362,0.168486,0.168486
fear,0.321755,0.290892,0.290892
in,0.186362,0.168486,0.168486
Sum,0.694479,0.627864,0.627864
